In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
#
from lrgsglib.core import *
#
move_to_rootf(print_tf=True)
plt.style.use('ipynb/nb_plotsheet.mplstyle')
#
path_plot = PATHPLOT / Path('PhD')
path_ene_plot_1 = path_plot / Path('ene_plot_1')
path_suppinf = PATHPLOT / Path(PATHNPAPR, 'suppinf')
path_extras = path_suppinf / Path('extras')
path_tsb = path_extras / Path('tsb')

Current working directory: /home/opisthofulax/PhD/projects/complex-networks/lrgsglib


### Energy plot

In [2]:
side = 64
pflip = 0.01
T = 0.1
pflip_list = [0.05, 0.08, 0.1, 0.2, 0.25, 0.35, 0.5]#[0.01, 0.05, 0.08, 0.1, 0.125, 0.2, 0.5]
T_list = np.linspace(1/side**1.5, 2.5, num=6)
seed = 0
cell = 'rand'
ic = "ground_state_0"
ic_list = ["uniform", "ground_state_0", "ground_state_1", "ground_state_2", "ground_state_10", "ground_state_50"]

In [4]:
ene = {}
s_out = {}
for pflip in pflip_list:
    lattice = Lattice2D(side, pflip=pflip, init_nw_dict=True, path_data=path_plot, seed=seed)
    lattice.flip_sel_edges(lattice.nwDict[cell]['G'])
    lattice.compute_k_eigvV(100)
    ene[pflip] = {}
    s_out[pflip] = {}
    for T in T_list:
        ene[pflip][T] = {}
        s_out[pflip][T] = {}
        for ic in ic_list:
            ic_shrt = ic.replace("ground_state_", "gs")
            outs = '_'.join([ic_shrt, cell, str(seed)])
            outs_plus = '_'.join([f"p={pflip:.3g}", f"T={T:.3g}", outs])
            sout_fname = lattice.path_ising / Path(f"sout_{outs_plus}.bin")
            ene_fname = lattice.path_ising / Path(f"ene_{outs_plus}.bin")
            if not (os.path.exists(sout_fname) and os.path.exists(ene_fname)):
                isdy = IsingDynamics(lattice, T=T, ic=ic, runlang="C3", out_suffix=f"{outs}")
                isdy.init_ising_dynamics()
                lattice.export_edgel_bin()
                isdy.export_ising_clust(which=1, val=-1)
                isdy.run(verbose=False, thrmSTEP=20)
            s_out[pflip][T][ic] = np.fromfile(sout_fname, dtype=np.int8).reshape(-1, *lattice.syshape)
            ene[pflip][T][ic] = np.fromfile(ene_fname, dtype=np.float64)*lattice.N/2

In [18]:
pflip_list, T_list

([0.01, 0.05, 0.08, 0.1, 0.125, 0.2, 0.5],
 array([1.95312500e-03, 5.01562500e-01, 1.00117188e+00, 1.50078125e+00,
        2.00039063e+00, 2.50000000e+00]))

In [54]:
figsize_ene = (8, 6*1.61)
figsize_sout = (8, 6)
eigV_col = credcblu
inset_color = '#ffe1db'
pflip = pflip_list[4]
T = T_list[0]
s_out_s = s_out[pflip][T]
ene_s = ene[pflip][T]

In [55]:
minene = 0
maxene = min(ene_s["ground_state_50"]/(2*lattice.N))
fig_ene, ax_ene = plt.subplots(figsize=figsize_ene)

yleft = False

ax_right = ax_ene.twinx()
if yleft:
    ax_ene.set_ylabel(r"$E/N$")
else:
    ax_right.set_ylabel(r"$E/N$")
    ax_ene.yaxis.set_ticks([])  # Remove ticks
    ax_ene.yaxis.set_ticklabels([])  # Remove labels
axplot = ax_ene if yleft else ax_right
ax_ene.set_xlabel(r"MC steps")

for ic in ic_list:
    if ic == 'uniform':
        ic_shrt = ic
        ic_label = 'random'
    else:
        ic_shrt = ic.replace("ground_state_", "gs")
        ic_numer = ic.replace("ground_state_", "")
        ic_label = fr"$\vec{{\sigma}}_0: \ket{{\lambda_{{{ic_numer}}}}}$"
    outs = '_'.join([ic_shrt, cell, str(seed)])
    outs_plus = '_'.join([f"p={pflip:.3g}", f"T={T:.3g}", outs])
    for i in [0, len(s_out_s[ic])//2, len(s_out_s[ic])-1]:
        fig, ax = plt.subplots(figsize=figsize_sout)
        ax.imshow(s_out_s[ic][i], cmap=eigV_col)
        ax.axis('off')
        fig.tight_layout()
        fig.savefig(path_ene_plot_1 / Path(f'IsingDynState{side}_{outs_plus}_t={i}.pdf'), 
                    bbox_inches='tight', transparent=True)
        plt.close(fig)
    x = np.arange(len(ene_s[ic]))
    indices = np.unique(np.logspace(0, np.log10(len(ene_s[ic]/(2*lattice.N))-1), num=1000, dtype=int))
    axplot.plot(x[indices], (ene_s[ic]/(2*lattice.N))[indices], 'k-' if ic == 'uniform' else '-', label=ic_label, zorder=100 if ic == 'uniform' else None)
    minene = min(minene, min(ene_s[ic]/(2*lattice.N)))
axplot.set_xscale('symlog')

zoomXrange = (80, len(ene_s["uniform"]))
zoomYrange = (1.005*minene, 0.95*maxene)  # Range of points to display in the inset
inset_ax = axplot.inset_axes([0.225, 0.25, 0.75, 0.3], 
                         xlim=zoomXrange, ylim=zoomYrange, fc=inset_color,
                         xticklabels=[], yticklabels=[], zorder=1000)  # Position and size of the inset

for ic in ic_list:
    x = np.arange(len(ene_s[ic]))
    indices = np.unique(np.logspace(0, np.log10(len(ene_s[ic]/(2*lattice.N))-1), num=1000, dtype=int))
    if ic == 'uniform':
        inset_ax.plot(x[indices], (ene_s[ic]/(2*lattice.N))[indices], 'k-', label=ic, zorder=100)
    else:
        inset_ax.plot(x[indices], (ene_s[ic]/(2*lattice.N))[indices], '-', label=ic)

mark_inset(axplot, inset_ax, loc1=4, loc2=3, fc=set_alpha_torgb(to_rgb(inset_color), 0.5), ec="k")
inset_ax.set_xscale('symlog')
inset_ax.set_xticklabels([])
# Enable minor ticks automatically
inset_ax.tick_params(axis='x', which='both', bottom=False, top=False, length=4)
inset_ax.tick_params(axis='y', which='both', left=False, right=False, length=4)

inset_ax.patch.set_alpha(1)
inset_ax.patch.set_facecolor(inset_color)
#
axplot.legend(ncol=2, fontsize=24)
fig_ene.savefig(path_ene_plot_1 / Path(f'IsingDynEne{side}_{pflip:.3g}_{T:.3g}_{ic}_{cell}.pdf'), 
            bbox_inches='tight', transparent=True)
plt.close('all')

In [51]:
pflip, T

(0.125, 1.001171875)

### Phase Diagram Sketch

In [19]:
lgeo = "3D"
J = 1
k_B = 1
match lgeo:
    case "2D":
        p_c = 0.103
        T_c = 2.27
        T_max = 2.5
    case "3D":
        p_c = 0.22
        T_c = 4.51
        T_max = 5
nishi_p = lambda T: 0.5 * (1 + np.exp(J / (k_B * T)))**(-1)
nishi_T = lambda p: J * (k_B*np.log(1/(2*p)-1))**(-1)
#
p_max = 0.5
TlinSpace = np.linspace(1e-2, T_max, 100)
pSglinSpace = np.linspace(p_c, p_max)
TSglinSpace = 0*np.ones(len(pSglinSpace))
#
critPoints = np.array([[0, p_c], [T_c, 0]])

In [20]:
kwNishiLine = dict(ls='-.', lw=1, marker='', c='k', clip_on=True)
kwCritPoints = dict(marker='o', c='r', clip_on=False, zorder=1e3)
kwSgLine = dict(lw=2, ls='-', c='#8030ff', marker='', clip_on=False, zorder=9e1)
#
fig, ax = plt.subplots(figsize=(8, 5))
#
# ax.plot(nishi_p(TlinSpace), TlinSpace, **kwNishiLine)
ax.scatter(*critPoints, **kwCritPoints)
ax.plot(pSglinSpace, TSglinSpace, **kwSgLine)
ax.set_ylim(0, T_max)
ax.set_xlim(0, p_max)
# |#
ax.set_xticks([0, p_c, 0.5])
ax.set_yticks([0, T_c])
ax.set_xticklabels([r"0", r"\(p_{\mathrm{c}}\)", r"0.5"])
ax.set_yticklabels([r"0", r"\(T_{\mathrm{c}}^{\text{\Large Ising}}\)"])
#
ax.tick_params(axis='both', which='both', labelsize=30)
#
# ax.text(p_max/2.25, T_max/2, r'\emph{Nishimori Line}', fontsize=16)
ax.text(p_c, 0.1, r'\emph{Spin Glass}', c='#8030ff', fontsize=20)
#
fig.savefig(path_plot / Path(f'phase_diagram_{lgeo}.pdf'), 
            bbox_inches='tight', transparent=True)
plt.close('all')

#### states

In [ ]:
match lgeo:
    case "2D":
        p_c = 0.103
        T_c = 2.27
        T_max = 2.5
    case "3D":
        p_c = 0.22
        T_c = 4.51
        T_max = 5

In [65]:
side = 48
pflip_list = [0.01, 0.05, 0.08, 0.12, 0.4]
T_list = np.linspace(side**(-.5), T_c, 5)
cell = 'rand'
ic = "uniform"

In [ ]:
for pflip in pflip_list:
    lattice = Lattice2D(side, pflip=pflip, init_nw_dict=True, path_data=path_plot, seed=0)
    lattice.flip_sel_edges(lattice.nwDict[cell]['G'])
    fig, ax = plt.subplots(ncols=len(T_list), figsize=(8, 8//len(T_list)))
    for i,T in enumerate(T_list):
        isdy = IsingDynamics(lattice, T=T, ic=ic, runlang="C3", out_suffix=f"{ic}_{cell}")
        isdy.init_ising_dynamics()
        lattice.export_edgel_bin()
        isdy.export_ising_clust(which=1, val=-1)
        isdy.run(verbose=False, thrmSTEP=20)
        # s_out = np.fromfile(lattice.path_ising / Path(f"sout_p={pflip:.3g}_T={T:.3g}_{ic}_{cell}.bin"), dtype=np.int8).reshape(-1, *lattice.syshape)
        ax[i].imshow(isdy.s.reshape(*lattice.syshape), cmap='gray', origin='lower')
        ax[i].axis('off')

In [82]:
side = 64
pflip = 0.1
cell = 'rand'
ic = "uniform"
T_list = np.linspace(side**(-.5), 2.5, 5)

In [85]:
lattice = Lattice2D(side, pflip=pflip, init_nw_dict=True, path_data=path_plot)
lattice.flip_sel_edges(lattice.nwDict[cell]['G'])
lattice.compute_k_eigvV(1)
fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(lattice.get_eigV_bin_check(0).reshape(*lattice.syshape), cmap='gray', origin='lower')
ax.axis('off')
fig.savefig(path_plot / Path(f'ising{side}eigV_{lgeo}_p={pflip:.3g}_T={T:.3g}_{ic}_{cell}.pdf'), 
            bbox_inches='tight', transparent=True)
for i,T in enumerate(T_list):
    isdy = IsingDynamics(lattice, T=T, ic=ic, runlang="C3", out_suffix=f"{ic}_{cell}")
    isdy.init_ising_dynamics()
    lattice.export_edgel_bin()
    isdy.run(verbose=False, thrmSTEP=20)
# s_out = np.fromfile(lattice.path_ising / Path(f"sout_p={pflip:.3g}_T={T:.3g}_{ic}_{cell}.bin"), dtype=np.int8).reshape(-1, *lattice.syshape)
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.imshow(isdy.s.reshape(*lattice.syshape), cmap='gray', origin='lower')
    ax.axis('off')
    fig.savefig(path_plot / Path(f'ising{side}_{lgeo}_p={pflip:.3g}_T={T:.3g}_{ic}_{cell}.pdf'), 
                bbox_inches='tight', transparent=True)
plt.close('all')

#### other states

In [5]:
side = 16
pflip = 0.01
T = 0.1
cell = 'rand'
ic = "ground_state_0"
ic_list = ["uniform", "ground_state_0", "ground_state_1", "ground_state_2", "ground_state_10", "ground_state_50"]

In [ ]:
lattice = Lattice2D(side, pflip=pflip, init_nw_dict=True, path_data=path_plot, seed=0)
lattice.flip_sel_edges(lattice.nwDict[cell]['G'])
lattice.compute_k_eigvV(100)

In [ ]:
ene = {}
for ic in ic_list:
    isdy = IsingDynamics(lattice, T=T, ic=ic, runlang="C3", out_suffix=f"{ic}_{cell}")
    isdy.init_ising_dynamics()
    lattice.export_edgel_bin()
    isdy.export_ising_clust(which=1, val=-1)
    isdy.run(verbose=False, thrmSTEP=20)
    s_out = np.fromfile(lattice.path_ising / Path(f"sout_p={pflip:.3g}_T={T:.3g}_{ic}_{cell}.bin"), dtype=np.int8).reshape(-1, *lattice.syshape)
    # ene = [compute_energy_sum(s.flatten(), lattice.gr['G'].edges(data='weight')) for s in s_out]
    ene[ic] = np.fromfile(lattice.path_ising / Path(f"ene_p={pflip:.3g}_T={T:.3g}_{ic}_{cell}.bin"), dtype=np.float64)*lattice.N/2
    fig, ax = plt.subplots(ncols=4, figsize=(12, 3))
    ax[0].imshow(s_out[0], cmap='binary')
    ax[0].axis('off')
    ax[1].imshow(s_out[len(s_out)//2], cmap='binary')
    ax[1].axis('off')
    ax[2].imshow(s_out[-1], cmap='binary')
    ax[2].axis('off')
    ax[3].plot(np.arange(1, len(ene[ic])+1), ene[ic])
    ax[3].set_xscale('log')
    fig.tight_layout()
    # fig.savefig(path_plot / Path(f'IsingDyn_{side}_{pflip:.3g}_{T:.3g}_{ic}_{cell}.pdf'), 
    #             bbox_inches='tight', transparent=True)